In [1]:
#Como los databases estan en archivos en sqlite, no necesito credenciales. Uso magic commands que usan sqlalchemy 
#pero no necesito importar sqlialchemy completa
import sqlite3
#para manipular data y llenar las tablas en sql
import pandas as pd
#para manipular archivos
import os

#para los magic commands si necesito prettytable
import prettytable
# Set the default display format for prettytable to 'DEFAULT' (i.e., a simple table format)
prettytable.DEFAULT = 'DEFAULT'
#cargo la extension para poder hacer magic commands
%load_ext sql

In [9]:
#si aun no existe, creo el database chicago, que tendra las tres tablas del proyecto
db_name = 'chicago.db'
db_exists = os.path.exists(db_name)
#si ya existe, se conecta, si no existia, lo crea y se conecta
conn = sqlite3.connect("chicago.db")
cur = conn.cursor()
#seteo al magic command para trabajar con este database 
%sql sqlite:///chicago.db

if db_exists:
    print("El database ya existia, y tiene las siguientes tablas:")
    res = %sql SELECT name FROM sqlite_master WHERE type='table';
    print(res)
else:
    print("El database no existia y fue creado")

El database ya existia, y tiene las siguientes tablas:
 * sqlite:///chicago.db
Done.
+----------------------------+
|            name            |
+----------------------------+
| chicago_socioeconomic_data |
|       school_chicago       |
|     chicago_crime_data     |
+----------------------------+


In [5]:
#creo las tres tablas, leyendolas de los csv en la carpeta data en este directorio
dir_data = ["./data/ChicagoCensusData.csv", "./data/ChicagoPublicSchools.csv", "./data/ChicagoCrimeData.csv"]
nombres_tablas = ['chicago_socioeconomic_data', 'school_chicago', 'chicago_crime_data']
for i in range(len(dir_data)):
    if os.path.exists(dir_data[i]):
        pd.read_csv(dir_data[i]).to_sql(nombres_tablas[i], conn, if_exists='replace', index=False)
        print(f"La tabla {nombres_tablas[i]} fue creada satisfactoriamente")
    else:
        print("La direccion de la data no existe")


La tabla chicago_socioeconomic_data fue creada satisfactoriamente
La tabla school_chicago fue creada satisfactoriamente
La tabla chicago_crime_data fue creada satisfactoriamente


In [8]:
%sql SELECT name FROM sqlite_master WHERE type='table';

 * sqlite:///chicago.db
Done.


name
chicago_socioeconomic_data
school_chicago
chicago_crime_data


### Analisis exploratorio de las tablas

In [10]:
#exploro las propiedades de las tablas en el database
for tabla in nombres_tablas:
    query = f"PRAGMA table_info('{tabla}');";
    res = %sql $query
    print(tabla)
    print(res)

 * sqlite:///chicago.db
Done.
chicago_socioeconomic_data
+-----+----------------------------------------------+---------+---------+------------+----+
| cid |                     name                     |   type  | notnull | dflt_value | pk |
+-----+----------------------------------------------+---------+---------+------------+----+
|  0  |            COMMUNITY_AREA_NUMBER             |   REAL  |    0    |    None    | 0  |
|  1  |             COMMUNITY_AREA_NAME              |   TEXT  |    0    |    None    | 0  |
|  2  |          PERCENT_OF_HOUSING_CROWDED          |   REAL  |    0    |    None    | 0  |
|  3  |       PERCENT_HOUSEHOLDS_BELOW_POVERTY       |   REAL  |    0    |    None    | 0  |
|  4  |         PERCENT_AGED_16__UNEMPLOYED          |   REAL  |    0    |    None    | 0  |
|  5  | PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA |   REAL  |    0    |    None    | 0  |
|  6  |       PERCENT_AGED_UNDER_18_OR_OVER_64       |   REAL  |    0    |    None    | 0  |
|  7  |      

### Problemas

1- Find the total number of crimes recorded in the CRIME table.

In [11]:
%sql SELECT COUNT(ID) FROM chicago_crime_data;

 * sqlite:///chicago.db
Done.


COUNT(ID)
533


3- List all case numbers for crimes involving minors?(children are not considered minors for the purposes of crime analysis)

In [13]:
#%sql SELECT * FROM chicago_crime_data WHERE primary_type LIKE '%CHILD%' OR description LIKE '%CHILD%';
%sql SELECT * FROM chicago_crime_data WHERE primary_type LIKE '%MINOR%' OR description LIKE '%MINOR%';

 * sqlite:///chicago.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
3987219,HL266884,2005-03-31,024XX N CLARK ST,2210,LIQUOR LAW VIOLATION,SELL/GIVE/DEL LIQUOR TO MINOR,CONVENIENCE STORE,1,0,2333,19,43.0,7.0,22,1172680.0,1916483.0,2005,41.92626872,-87.64089934,"(41.926268719, -87.640899336)"
3266814,HK238408,2004-03-13,093XX S STONY ISLAND AVE,2230,LIQUOR LAW VIOLATION,ILLEGAL CONSUMPTION BY MINOR,ALLEY,1,0,413,4,8.0,48.0,22,1188539.0,1843379.0,2004,41.72530099,-87.58496589,"(41.72530099, -87.584965887)"
